# **POLAR @ SemEval-2026 — Task9**
## Subtask 1: Polarization Detection    
### Authors: Sujay Nalimela, Vedant Kesharia, Gaurav Shinde
### Date: December 2025

---

This notebook contains the *full pipeline* for all POLAR shared task subtasks:

### ✔ Subtask 1 — Polarization Detection (binary)  
### ✔ Subtask 2 — Polarization Type Classification (multi-class)  
### ✔ Subtask 3 — Polarization Manifestation Identification

**Task Overview:**
- **Subtask 1**: Binary classification to detect if text contains polarization
- **Subtask 2**: Multi-label classification to identify types of polarization (political, racial/ethnic, religious, gender/sexual, other)
- **Subtask 3**: Multi-label classification to identify how polarization manifests (stereotype, vilification, dehumanization, extreme_language, lack_of_empathy, invalidation)

## Environment Setup and Cleanup

Clean up working directory and cache to ensure fresh start and avoid conflicts.

In [1]:
# rm -rf /kaggle/working/*

In [2]:
# rm -rf /root/.cache/huggingface

In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [2]:
!git clone https://github.com/sujaynalimela/polar-semeval-2026-task9.git

Cloning into 'polar-semeval-2026-task9'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 39 (delta 11), reused 16 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 19.57 KiB | 4.89 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
import sys
sys.path.append("/kaggle/working/polar-semeval-2026-task9")

print(sys.path)


['/kaggle/working', '/kaggle/lib/kagglegym', '/kaggle/lib', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/kaggle/working/polar-semeval-2026-task9']


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    set_seed,
)

from shared.preprocess import clean_text
from shared.metrics import macro_f1

torch.backends.mps.allow_tf32 = True

print("Torch version:", torch.__version__)
set_seed(42)


2025-12-08 01:16:11.700071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765156571.891858      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765156571.951828      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Torch version: 2.6.0+cu124


We begin with **Subtask 1 (binary)** using **XLM-RoBERTa-Base**.

## ⚙️ Configuration and Hyperparameters

Set all training parameters, paths, and model configuration:
- **Model**: LaBSE (Language-agnostic BERT Sentence Embedding), also tried on bert, indicbert, muril and xlm roberta models
- **Language**: Current language Chinese (zho) - also tried on hindi and english
- **Max Length**: 224 tokens for efficient training
- **Training**: 10 epochs with learning rate 2e-5
- **Hardware**: GPU (P100)

In [10]:
BASE_DATA = "/kaggle/input/datasetphase2/data2"
LANG = "zho"  # change to arb, spa, amh, etc.

MODEL_NAME = "sentence-transformers/LaBSE"
MAX_LEN = 224
EPOCHS = 10
BATCH_SIZE = 4
GRAD_ACCUM = 2
LR = 2e-5
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01

# Subtask 1 overrides
SUBTASK1_EPOCHS = 10
SUBTASK1_LR = 2e-5

# Subtask 2 overrides
SUBTASK2_EPOCHS = 10
SUBTASK2_LR = 2e-5

# Subtask 3 overrides
SUBTASK3_EPOCHS = 5
SUBTASK3_LR = 1.5e-5


## **Load Dataset**

Each language includes:

- `train/<lang>.csv`  
- `dev/<lang>.csv`  
- `test/<lang>.csv`  

We read all splits and inspect the schema.

In [11]:
train_path = f"{BASE_DATA}/subtask1/train/{LANG}.csv"
dev_path   = f"{BASE_DATA}/subtask1/dev/{LANG}.csv"

train_df = pd.read_csv(train_path)
dev_df   = pd.read_csv(dev_path)

print("Train size:", len(train_df))
print("Dev size:", len(dev_df))

train_df.head()


Train size: 4280
Dev size: 214


,id,text,polarization
0,zho_e59ce789a1d83d91cb7d362efaa3bf23,好久都没有见过那么干净的白人班级了，一个黑乐色都没有,1
1,zho_ea3ef5567698fc40fa631baf7f697a16,楼主这讽刺太有力度把跪虫族和国际鬼子虫类及其它们后代虫混描写淋漓尽致,1
2,zho_b73779673957f6dab5688c27a6747458,你这样说让那些标榜找个外国对象倍有面的人还怎么装b�,1
3,zho_f7164a14baafbcb6ec387b0217a1c766,说明一下，我只关爱自己身边的女性，比如老婆女儿等，别的女人关我鸟事。v,1
4,zho_1be506c572b805494787f548671a5bb7,基佬还说之前视频里的不是它，它都说照片是它爆出来的了。,1


## **Preprocessing**

We apply light normalization (lowercasing, URL removal, emoji normalization)
using our shared `clean_text()` function.


## 🔄 Text Preprocessing

Apply text cleaning pipeline:
- Lowercase normalization
- URL removal
- Emoji standardization
- Special character handling

In [12]:
train_df["text"] = train_df["text"].apply(clean_text)
dev_df["text"]   = dev_df["text"].apply(clean_text)


In [13]:
import numpy as np

# def tune_binary_threshold(logits, labels, grid=None):
#     grid = grid or np.linspace(0.1, 0.9, 17)
#     probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
#     best_f1, best_t = -1, 0.5
#     for t in grid:
#         preds = (probs >= t).astype(int)
#         score = f1_score(labels, preds)
#         if score > best_f1:
#             best_f1, best_t = score, t
#     return best_t, best_f1

def tune_binary_threshold(logits, labels, grid=None):
    grid = grid or np.linspace(0.1, 0.9, 17)
    probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    best_f1, best_t = -1, 0.5
    for t in grid:
        preds = (probs >= t).astype(int)
        score = f1_score(labels, preds, average="macro")
        if score > best_f1:
            best_f1, best_t = score, t
    return best_t, best_f1

def tune_multilabel_threshold(logits, labels, grid=None):
    grid = grid or np.linspace(0.1, 0.8, 15)
    probs = 1 / (1 + np.exp(-logits))
    best_f1, best_t = -1, 0.5
    for t in grid:
        preds = (probs >= t).astype(int)
        score = f1_score(labels, preds, average="macro", zero_division=0)
        if score > best_f1:
            best_f1, best_t = score, t
    return best_t, best_f1

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        base_model = model.module if hasattr(model, "module") else model
        outputs = model(**inputs)
        if self.class_weights is not None:
            device = next(model.parameters()).device
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(device))
            loss = loss_fct(outputs.logits.view(-1, base_model.config.num_labels), labels.view(-1))
        else:
            loss = outputs.loss
        return (loss, outputs) if return_outputs else loss


class BCETrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        base_model = model.module if hasattr(model, "module") else model
        device = next(model.parameters()).device
        loss_fct = nn.BCEWithLogitsLoss(
            pos_weight=self.pos_weight.to(device) if self.pos_weight is not None else None
        )
        loss = loss_fct(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

# class weights for Subtask 1
pos_frac = train_df["polarization"].mean()
class_weights = torch.tensor([1 - pos_frac, pos_frac], dtype=torch.float)
print("Class weights (neg,pos):", class_weights.tolist())


Class weights (neg,pos): [0.5044392347335815, 0.49556073546409607]


In [14]:
train_df_split, val_df_split = train_test_split(
    train_df,
    test_size=0.1,
    stratify=train_df["polarization"],
    random_state=42
)
train_dataset = Dataset.from_pandas(train_df_split)
val_dataset   = Dataset.from_pandas(val_df_split)


## **Tokenization**

We use a single tokenizer across all languages (multilingual model).

Convert text to model inputs using the LaBSE tokenizer:
- Truncate sequences to MAX_LEN tokens
- Pad shorter sequences to MAX_LEN
- Generate attention masks
- Prepare for PyTorch Trainer

The tokenizer handles multiple scripts (Latin, Devanagari, Arabic, etc.) seamlessly.


In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

# tokenization
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset   = val_dataset.map(tokenize, batched=True)

# rename correct label column
train_dataset = train_dataset.rename_column("polarization", "labels")
val_dataset   = val_dataset.rename_column("polarization", "labels")

# set PyTorch formatting
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3852 [00:00<?, ? examples/s]

Map:   0%|          | 0/428 [00:00<?, ? examples/s]

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/LaBSE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 🚀 Train Subtask 1 Model

Train the binary classification model using WeightedTrainer with class weights.

**Training Process:**
1. Forward pass with mixed precision
2. Compute weighted cross-entropy loss
3. Backward pass with gradient accumulation
4. Update weights every 2 gradient accumulation steps
5. Evaluate on validation set after each epoch
6. Save best model based on macro F1

**Expected Training Time**: ~15-25 minutes on GPU

In [ ]:
training_args = TrainingArguments(
    output_dir="/kaggle/temp/subtask1",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=SUBTASK1_LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    gradient_accumulation_steps=GRAD_ACCUM,
    # per_device_train_batch_size=2,
    # per_device_eval_batch_size=16,
    # gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    fp16=True,
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    logging_steps=50,
    report_to="none",
)


from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    preds = (probs >= 0.5).astype(int)
    return {"macro_f1": f1_score(labels, preds, average="macro")}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    class_weights=class_weights,
)
trainer.train()


/tmp/ipykernel_47/506938794.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss


In [ ]:
val_pred = trainer.predict(val_dataset)
best_threshold_sub1, best_val_f1_sub1 = tune_binary_threshold(val_pred.predictions, val_pred.label_ids)
print(f"Best val threshold (Subtask 1): {best_threshold_sub1:.2f} | F1={best_val_f1_sub1:.4f}")


In [18]:
# full_train_dataset = Dataset.from_pandas(train_df)

# full_train_dataset = full_train_dataset.map(tokenize, batched=True)
# full_train_dataset = full_train_dataset.rename_column("polarization", "labels")
# full_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# full_training_args = TrainingArguments(
#     output_dir=f"subtask1/{LANG}_checkpoints",
#     eval_strategy="no",       # ⛔ Disable evaluation
#     save_strategy="no",       # Optional: don’t save intermediate checkpoints
#     learning_rate=3e-5,
#     per_device_train_batch_size=4,
#     num_train_epochs=2,
#     logging_steps=50,
#     report_to="none"
# )

# trainer = Trainer(
#     model=model,
#     args=full_training_args,
#     train_dataset=full_train_dataset,
#     tokenizer=tokenizer,
# )

# trainer.train()


In [17]:
# -------------------- Load official test data (dev/) --------------------

dev_pred_path = f"{BASE_DATA}/dev/{LANG}.csv"
test_df = pd.read_csv(dev_pred_path)

print("Loaded test (dev) size:", len(test_df))
test_df.head()

Loaded test (dev) size: 137


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,text,polarization
0,hin_4e26c967ea5fd015078d088118fb623c,और इस तरह बीजेपी ने अपना इस्लामीकरण कर लिया,NaN
1,hin_ad83fddd1620ad95784072561ea3d3d0,ओरीजनल हिंदुत्व यही है ।,NaN
2,hin_21522a5233b239644c874f8b52890d70,हिंदुत्व की चर्बी उतर गई,NaN
3,hin_b7fa87154f21b0abd1eac2c3973c7812,हिंदू एवम हिंदुत्व जाग रहा है। जय श्री राम,NaN
4,hin_d3e4ed943e52657cc4ad3c76d6747d60,बोल अपने आका को हिन्दू और हिंदुत्व है .......घ...,NaN


In [18]:
# Clean text using your preprocessing function
test_df["text"] = test_df["text"].apply(clean_text)


In [19]:
# Prepare inputs
inputs = tokenizer(
    test_df["text"].tolist(),
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
    return_tensors="pt"
)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}


In [20]:
# Disable gradient calculation for faster inference
with torch.no_grad():
    logits = model(**inputs).logits

probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
preds = (probs >= best_threshold_sub1).astype(int)


In [21]:
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "polarization": preds.astype(int)
})

submission_df.head()


,id,polarization
0,hin_4e26c967ea5fd015078d088118fb623c,1
1,hin_ad83fddd1620ad95784072561ea3d3d0,1
2,hin_21522a5233b239644c874f8b52890d70,1
3,hin_b7fa87154f21b0abd1eac2c3973c7812,1
4,hin_d3e4ed943e52657cc4ad3c76d6747d60,1


In [22]:
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "polarization": preds.astype(int)
})

submission_df.head()


,id,polarization
0,hin_4e26c967ea5fd015078d088118fb623c,1
1,hin_ad83fddd1620ad95784072561ea3d3d0,1
2,hin_21522a5233b239644c874f8b52890d70,1
3,hin_b7fa87154f21b0abd1eac2c3973c7812,1
4,hin_d3e4ed943e52657cc4ad3c76d6747d60,1


In [23]:
import os

SAVE_DIR = "submissions/subtask1"
os.makedirs(SAVE_DIR, exist_ok=True)

OUT_PATH = f"{SAVE_DIR}/pred_{LANG}.csv"
submission_df.to_csv(OUT_PATH, index=False)

print("Saved prediction file:", OUT_PATH)


Saved prediction file: submissions/subtask1/pred_hin.csv


In [24]:
import shutil

zip_name = "subtask_1"   # final file will be submission_subtask1.zip
shutil.make_archive(zip_name, "zip", SAVE_DIR)

print("Created ZIP:", zip_name + ".zip")


Created ZIP: subtask_1.zip


# ----------------------------------------------
# SUBTASK 2 — Polarization Type Classification
# ----------------------------------------------

This section implements **Subtask 2** of POLAR @ SemEval-2026.

### ✔ Objective  
Given a text, predict which *types* of polarization apply:

- `political`
- `racial/ethnic`
- `religious`
- `gender/sexual`
- `other`

This is a **multi-label classification task** → each text may have multiple 1s.

### ✔ Notes
- Subtask 2 is trained **independently** from Subtask 1.
- Dataset contains **only train + test**, so we split train into *train/validation*.
- We reuse the same multilingual model (`bert-base-multilingual-cased`) as Subtask 1.


In [11]:
print("=== SUBTASK 2: Polarization Type Classification ===")

sub2_path = f"/kaggle/input/datasetphase2/data2/subtask2/train/{LANG}.csv"
df2 = pd.read_csv(sub2_path)

print("Subtask 2 train size:", len(df2))
df2.head()

=== SUBTASK 2: Polarization Type Classification ===
Subtask 2 train size: 4280


,id,text,political,racial/ethnic,religious,gender/sexual,other
0,zho_e59ce789a1d83d91cb7d362efaa3bf23,好久都没有见过那么干净的白人班级了，一个黑乐色都没有,0,1,0,0,0
1,zho_ea3ef5567698fc40fa631baf7f697a16,楼主这讽刺太有力度把跪虫族和国际鬼子虫类及其它们后代虫混描写淋漓尽致,0,1,0,0,0
2,zho_b73779673957f6dab5688c27a6747458,你这样说让那些标榜找个外国对象倍有面的人还怎么装b�,0,1,0,0,0
3,zho_f7164a14baafbcb6ec387b0217a1c766,说明一下，我只关爱自己身边的女性，比如老婆女儿等，别的女人关我鸟事。v,0,0,0,1,0
4,zho_1be506c572b805494787f548671a5bb7,基佬还说之前视频里的不是它，它都说照片是它爆出来的了。,0,0,0,1,0


## Preprocess text & prepare label columns

We apply the same preprocessing (`clean_text`) used in Subtask 1.  
Subtask 2 has **5 binary label columns**:

- `political`
- `racial/ethnic`
- `religious`
- `gender/sexual`
- `other`

We convert them to integer (0/1), fill missing values, and create a **multi-label target matrix**.


In [12]:
# Subtask 2 label names (update if more languages)
SUBTASK2_LABELS = [
    "political",
    "racial/ethnic",
    "religious",
    "gender/sexual",
    "other"
]


## Load Subtask-2 English Data

Train path and dev path correspond to **train.csv** and **dev.csv** for Subtask 2.
IMPORTANT:
- Dev = test split (no gold labels for submission)
- To evaluate locally, we will create a validation split from the train set.


In [13]:
train2_path = f"{BASE_DATA}/subtask2/train/{LANG}.csv"
test2_path  = f"{BASE_DATA}/subtask2/dev/{LANG}.csv"

train2_df = pd.read_csv(train2_path)
test2_df  = pd.read_csv(test2_path)

print(train2_df.head())
print("Train size:", len(train2_df))
print("Test size:", len(test2_df))


                                     id                                 text  \
0  zho_e59ce789a1d83d91cb7d362efaa3bf23           好久都没有见过那么干净的白人班级了，一个黑乐色都没有   
1  zho_ea3ef5567698fc40fa631baf7f697a16   楼主这讽刺太有力度把跪虫族和国际鬼子虫类及其它们后代虫混描写淋漓尽致   
2  zho_b73779673957f6dab5688c27a6747458           你这样说让那些标榜找个外国对象倍有面的人还怎么装b�   
3  zho_f7164a14baafbcb6ec387b0217a1c766  说明一下，我只关爱自己身边的女性，比如老婆女儿等，别的女人关我鸟事。v   
4  zho_1be506c572b805494787f548671a5bb7          基佬还说之前视频里的不是它，它都说照片是它爆出来的了。   

   political  racial/ethnic  religious  gender/sexual  other  
0          0              1          0              0      0  
1          0              1          0              0      0  
2          0              1          0              0      0  
3          0              0          0              1      0  
4          0              0          0              1      0  
Train size: 4280
Test size: 214


### Preprocess and create train/validation split  
We will mimic Subtask-1 behavior by splitting the official train set into:  
- **train** (80%)  
- **validation** (20%)  

We also convert label columns to float32.


In [14]:
# Clean text
train2_df["text"] = train2_df["text"].apply(clean_text)
test2_df["text"]  = test2_df["text"].apply(clean_text)

# Train/validation split
train2_df, val2_df = train_test_split(train2_df, test_size=0.2, random_state=42)

# Convert multi-label columns to float32
train2_df[SUBTASK2_LABELS] = train2_df[SUBTASK2_LABELS].astype("float32")
val2_df[SUBTASK2_LABELS]   = val2_df[SUBTASK2_LABELS].astype("float32")

# Pos weight for BCE (handle imbalance)
pos_counts2 = train2_df[SUBTASK2_LABELS].sum().values
neg_counts2 = len(train2_df) - pos_counts2
pos_weight2 = torch.tensor(neg_counts2 / (pos_counts2 + 1e-6), dtype=torch.float)
pos_weight2 = torch.clamp(pos_weight2, max=5.0)
print("Subtask2 pos_weight (capped):", pos_weight2.tolist())


Subtask2 pos_weight (capped): [5.0, 3.4467532634735107, 5.0, 5.0, 5.0]


### Convert to HuggingFace Dataset  
We must produce a **single combined `labels` vector** per row for multi-label training.


In [15]:
def add_labels_sub2(example):
    example["labels"] = [float(example[col]) for col in SUBTASK2_LABELS]
    return example

train2_ds = Dataset.from_pandas(train2_df)
val2_ds   = Dataset.from_pandas(val2_df)
test2_ds  = Dataset.from_pandas(test2_df)  # for final predictions

train2_ds = train2_ds.map(add_labels_sub2)
val2_ds   = val2_ds.map(add_labels_sub2)


Map:   0%|          | 0/3424 [00:00<?, ? examples/s]

Map:   0%|          | 0/856 [00:00<?, ? examples/s]

### Tokenize using the same tokenizer as Subtask-1  
We reuse the tokenizer defined earlier (bert-base-multilingual-cased).


In [16]:
train2_ds = train2_ds.map(tokenize, batched=True)
val2_ds   = val2_ds.map(tokenize, batched=True)
test2_ds  = test2_ds.map(tokenize, batched=True)


Map:   0%|          | 0/3424 [00:00<?, ? examples/s]

Map:   0%|          | 0/856 [00:00<?, ? examples/s]

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

### Set PyTorch format  
Only include:
- input_ids  
- attention_mask  
- labels  


In [18]:
train2_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)
val2_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

test2_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"]
)


### Initialize Multilingual BERT for Multi-Label Classification


In [19]:
model2 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(SUBTASK2_LABELS),
    problem_type="multi_label_classification"
)


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/LaBSE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### TrainingArguments  
We reuse the same training parameters as Subtask-1.


In [20]:
training_args2 = TrainingArguments(
    output_dir="/kaggle/temp/subtask2",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=SUBTASK2_LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    gradient_accumulation_steps=GRAD_ACCUM,
    # per_device_train_batch_size=4,
    # per_device_eval_batch_size=16,
    # gradient_accumulation_steps=2,
    num_train_epochs=SUBTASK2_EPOCHS,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    fp16=True,
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    logging_steps=50,
    report_to="none",
)


### Macro F1 across all 5 labels  
Threshold applied after sigmoid.


In [21]:
def compute_metrics_sub2(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs > 0.5).astype(int)
    score = f1_score(labels, preds, average="macro", zero_division=0)
    return {"macro_f1": float(score)}


### Train Subtask-2 Model


In [22]:
# from IPython.display import FileLink

# FileLink('submissions/subtask1/pred_eng.csv')


/kaggle/working/submissions/subtask1/pred_eng.csv

In [23]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [24]:
trainer2 = BCETrainer(
    model=model2,
    args=training_args2,
    train_dataset=train2_ds,
    eval_dataset=val2_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_sub2,
    pos_weight=pos_weight2,
)

trainer2.train()


/tmp/ipykernel_47/506938794.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `BCETrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Macro F1
1,0.379600,0.425677,0.688698
2,0.320300,0.413392,0.721575
3,0.233100,0.435285,0.723637
4,0.131300,0.529567,0.736052
5,0.071400,0.629575,0.739375
6,0.064600,0.645438,0.717180
7,0.033300,0.694576,0.744327
8,0.010300,0.675540,0.751165
9,0.023700,0.741399,0.745970
10,0.007100,0.741807,0.744971


TrainOutput(global_step=4280, training_loss=0.1459432421478434, metrics={'train_runtime': 1730.8391, 'train_samples_per_second': 19.782, 'train_steps_per_second': 2.473, 'total_flos': 3941509774110720.0, 'train_loss': 0.1459432421478434, 'epoch': 10.0})

In [ ]:
# from IPython.display import FileLink

# FileLink(f'submissions/subtask2/pred_{LANG}.csv')


In [25]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


### Generate predictions for submission  
(You will combine these later inside `subtask_2/pred_eng.csv`)


In [ ]:
# def tune_per_label_threshold(logits, labels, grid=None):
#     grid = grid or np.linspace(0.1, 0.8, 15)
#     probs = 1 / (1 + np.exp(-logits))
#     best = []
#     for i in range(labels.shape[1]):
#         best_f1, best_t = -1, 0.5
#         for t in grid:
#             preds = (probs[:, i] >= t).astype(int)
#             f1 = f1_score(labels[:, i], preds, zero_division=0)
#             if f1 > best_f1:
#                 best_f1, best_t = f1, t
#         best.append(best_t)
#     return best

In [26]:
from sklearn.metrics import f1_score

def tune_macro_threshold(logits, labels, grid=None):
    grid = grid or np.linspace(0.1, 0.8, 15)
    probs = 1 / (1 + np.exp(-logits))
    best_f1, best_t = -1, 0.5
    for t in grid:
        preds = (probs >= t).astype(int)
        f1 = f1_score(labels, preds, average="macro", zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t, best_f1

# Tune threshold on validation (after trainer2.train(), using the best checkpoint)
val_pred2 = trainer2.predict(val2_ds)
best_t, best_f1 = tune_macro_threshold(val_pred2.predictions, val_pred2.label_ids)
print(f"Best macro threshold: {best_t:.2f} | F1={best_f1:.4f}")

# Predict on test/dev with that threshold
pred_logits = trainer2.predict(test2_ds).predictions
pred_probs = 1 / (1 + np.exp(-pred_logits))
pred_labels = (pred_probs >= best_t).astype(int)

sub2_out = pd.DataFrame({"id": test2_df["id"]})
for i, col in enumerate(SUBTASK2_LABELS):
    sub2_out[col] = pred_labels[:, i]

sub2_out.head()

Best macro threshold: 0.30 | F1=0.7560


,id,political,racial/ethnic,religious,gender/sexual,other
0,zho_3653010632292449856c11f234040704,0,1,0,0,0
1,zho_f186c48ab10bbad0110cc57f81929208,0,0,0,0,1
2,zho_cc96cfb2e302edf570548253578ced89,0,0,0,0,1
3,zho_6d06ab3b817c68c5bab057add5b06447,0,1,0,0,0
4,zho_ffe6eda7b6330733276218813605d369,0,1,0,0,0


In [27]:
import os

SAVE_DIR = "submissions/subtask2"
os.makedirs(SAVE_DIR, exist_ok=True)

out_path = f"{SAVE_DIR}/pred_{LANG}.csv"
sub2_out.to_csv(out_path, index=False)
out_path


'submissions/subtask2/pred_zho.csv'

# -----------------------------------------------------------
# Subtask 3 — Manifestation Identification (Multi-Label)
# -----------------------------------------------------------

Subtask 3 predicts **how** polarization is expressed in the text.

Labels in this subtask:

- stereotype  
- vilification  
- dehumanization  
- extreme_language  
- lack_of_empathy  
- invalidation

This is a **multi-label classification** task:
a single text may contain multiple manifestations.

As before, we will:

- preprocess text  
- split train → train/val  
- build a single multi-label `labels` vector  
- train a Transformer with BCEWithLogitsLoss  
- evaluate using Macro-F1  
- generate predictions for Codabench submission  


In [16]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [17]:
SUBTASK3_LABELS = [
    "stereotype",
    "vilification",
    "dehumanization",
    "extreme_language",
    "lack_of_empathy",
    "invalidation"
]


### Load Subtask-3 English Data

Same structure as previous subtasks:

- train/eng.csv  
- dev/eng.csv (this is the test set for submission)

We will create our own validation split from the train set.


In [18]:
train3_path = f"{BASE_DATA}/subtask3/train/{LANG}.csv"
test3_path  = f"{BASE_DATA}/subtask3/dev/{LANG}.csv"

train3_df = pd.read_csv(train3_path)
test3_df  = pd.read_csv(test3_path)

print(train3_df.head())
print("Train size:", len(train3_df))
print("Test size:", len(test3_df))


                                     id                                 text  \
0  zho_e59ce789a1d83d91cb7d362efaa3bf23           好久都没有见过那么干净的白人班级了，一个黑乐色都没有   
1  zho_ea3ef5567698fc40fa631baf7f697a16   楼主这讽刺太有力度把跪虫族和国际鬼子虫类及其它们后代虫混描写淋漓尽致   
2  zho_b73779673957f6dab5688c27a6747458           你这样说让那些标榜找个外国对象倍有面的人还怎么装b�   
3  zho_f7164a14baafbcb6ec387b0217a1c766  说明一下，我只关爱自己身边的女性，比如老婆女儿等，别的女人关我鸟事。v   
4  zho_1be506c572b805494787f548671a5bb7          基佬还说之前视频里的不是它，它都说照片是它爆出来的了。   

   stereotype  vilification  dehumanization  extreme_language  \
0           1             1               1                 0   
1           0             1               1                 0   
2           1             0               0                 0   
3           0             0               0                 0   
4           0             1               0                 0   

   lack_of_empathy  invalidation  
0                0             0  
1                0             0  
2                0     

### Preprocess text & split dataset  
We follow the same approach as Subtask-1 & Subtask-2:

- Clean text using `clean_text()`
- Split train into 80% train, 20% validation
- Convert the label columns to float32


In [19]:
# Clean text
train3_df["text"] = train3_df["text"].apply(clean_text)
test3_df["text"]  = test3_df["text"].apply(clean_text)

# Train/validation split
train3_df, val3_df = train_test_split(train3_df, test_size=0.2, random_state=42)

# Convert labels to float32
train3_df[SUBTASK3_LABELS] = train3_df[SUBTASK3_LABELS].astype("float32")
val3_df[SUBTASK3_LABELS]   = val3_df[SUBTASK3_LABELS].astype("float32")

# Pos weight for BCE
pos_counts3 = train3_df[SUBTASK3_LABELS].sum().values
neg_counts3 = len(train3_df) - pos_counts3
pos_weight3 = torch.tensor(neg_counts3 / (pos_counts3 + 1e-6), dtype=torch.float)
print("Subtask3 pos_weight:", pos_weight3.tolist())


Subtask3 pos_weight: [2.3242719173431396, 4.400630950927734, 20.006134033203125, 11.588234901428223, 12.220077514648438, 20.267080307006836]


### Convert to HuggingFace Dataset  
We build a single `labels` vector for multi-label classification.


In [20]:
def add_labels_sub3(example):
    example["labels"] = [float(example[col]) for col in SUBTASK3_LABELS]
    return example

train3_ds = Dataset.from_pandas(train3_df)
val3_ds   = Dataset.from_pandas(val3_df)
test3_ds  = Dataset.from_pandas(test3_df)

train3_ds = train3_ds.map(add_labels_sub3)
val3_ds   = val3_ds.map(add_labels_sub3)


Map:   0%|          | 0/3424 [00:00<?, ? examples/s]

Map:   0%|          | 0/856 [00:00<?, ? examples/s]

### Tokenize using the same tokenizer  
Using identical settings ensures consistency across subtasks.


In [21]:
# train3_ds = train3_ds.map(tokenize, batched=True)
# val3_ds   = val3_ds.map(tokenize, batched=True)
# test3_ds  = test3_ds.map(tokenize, batched=True)


MAX_LEN = 160  # temporary for Subtask 3
train3_ds = train3_ds.map(tokenize, batched=True)
val3_ds   = val3_ds.map(tokenize, batched=True)
test3_ds  = test3_ds.map(tokenize, batched=True)
MAX_LEN = 224  # restore if needed later


Map:   0%|          | 0/3424 [00:00<?, ? examples/s]

Map:   0%|          | 0/856 [00:00<?, ? examples/s]

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

### Set dataset format for Trainer  
Only include:

- input_ids  
- attention_mask  
- labels  


In [22]:
train3_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)
val3_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

test3_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"]
)


### Initialize Transformer for Multi-Label Classification


In [23]:
model3 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(SUBTASK3_LABELS),
    problem_type="multi_label_classification"
)


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/LaBSE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### TrainingArguments  
We reuse the same hyperparameters as Subtask-1 and 2.


In [24]:
training_args3 = TrainingArguments(
    output_dir="/kaggle/temp/subtask3",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=SUBTASK3_LR,
    
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=32,
    # gradient_accumulation_steps=4,
    
    # per_device_train_batch_size=4,
    # per_device_eval_batch_size=16,
    # gradient_accumulation_steps=4,
    
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    
    num_train_epochs=SUBTASK3_EPOCHS,
    warmup_ratio=WARMUP_RATIO,
    # weight_decay=WEIGHT_DECAY,
    weight_decay=0.02,
    fp16=True,
    gradient_checkpointing=True,
    label_smoothing_factor=0.05,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    logging_steps=50,
    report_to="none",
)


### Macro F1 across all 6 labels  
Using sigmoid activation + threshold 0.5


In [25]:
def compute_metrics_sub3(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs > 0.5).astype(int)
    score = f1_score(labels, preds, average="macro", zero_division=0)
    return {"macro_f1": float(score)}


### Train the Subtask-3 Model


In [26]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer3 = BCETrainer(
    model=model3,
    args=training_args3,
    train_dataset=train3_ds,
    eval_dataset=val3_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_sub3,
    pos_weight=pos_weight3,
)

trainer3.train()


/tmp/ipykernel_47/506938794.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `BCETrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Macro F1
1,0.819500,0.960700,0.458859
2,0.682000,0.834463,0.487733
3,0.454400,0.976924,0.538017


### Generate predictions for submission  
We apply sigmoid + 0.5 thresholding.


In [ ]:
# val_pred3 = trainer3.predict(val3_ds)
# best_threshold_sub3, best_val_f1_sub3 = tune_multilabel_threshold(val_pred3.predictions, val_pred3.label_ids)
# print(f"Best val threshold (Subtask 3): {best_threshold_sub3:.2f} | F1={best_val_f1_sub3:.4f}")

# pred_logits_3 = trainer3.predict(test3_ds).predictions
# pred_probs_3 = 1 / (1 + np.exp(-pred_logits_3))
# pred_labels_3 = (pred_probs_3 >= best_threshold_sub3).astype(int)

# sub3_out = pd.DataFrame({
#     "id": test3_df["id"],
# })

# for i, col in enumerate(SUBTASK3_LABELS):
#     sub3_out[col] = pred_labels_3[:, i]

# sub3_out.head()


from sklearn.metrics import f1_score

def tune_macro_threshold(logits, labels, grid=None):
    grid = grid or np.linspace(0.1, 0.8, 15)
    probs = 1 / (1 + np.exp(-logits))
    best_f1, best_t = -1, 0.5
    for t in grid:
        preds = (probs >= t).astype(int)
        f1 = f1_score(labels, preds, average="macro", zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t, best_f1

val_pred3 = trainer3.predict(val3_ds)
best_t3, best_f1_3 = tune_macro_threshold(val_pred3.predictions, val_pred3.label_ids)
print(f"Best macro threshold (Subtask 3): {best_t3:.2f} | F1={best_f1_3:.4f}")

pred_logits_3 = trainer3.predict(test3_ds).predictions
pred_probs_3 = 1 / (1 + np.exp(-pred_logits_3))
pred_labels_3 = (pred_probs_3 >= best_t3).astype(int)

sub3_out = pd.DataFrame({"id": test3_df["id"]})
for i, col in enumerate(SUBTASK3_LABELS):
    sub3_out[col] = pred_labels_3[:, i]

sub3_out.head()


In [ ]:
import os

SAVE_DIR = "submissions/subtask3"
os.makedirs(SAVE_DIR, exist_ok=True)

out_path3 = f"{SAVE_DIR}/pred_{LANG}.csv"
sub3_out.to_csv(out_path3, index=False)
out_path3
